In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.styles import Font
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import time
import datetime
import re
import math

In [18]:
DRIVER_PATH = r"chromedriver"
print("**** INITIALIZING DRIVER ****")
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
url="https://unifiedportal-epfo.epfindia.gov.in/publicPortal/no-auth/misReport/home/loadEstSearchHome"
driver.get(url)
time.sleep(3)

**** INITIALIZING DRIVER ****


In [38]:
import selenium
selenium.__version__

'3.141.0'

In [19]:
COMPANY_NAME = input("Enter company name: ")

# EST_ID = input("Establishment Code: ")

# est = driver.find_element_by_id("estCode")
# est.clear()
# est.send_keys(EST_ID)

name_of_establishment = driver.find_element_by_id("estName")
name_of_establishment.clear()
name_of_establishment.send_keys(COMPANY_NAME)

captacha = driver.find_element_by_id("captcha")
captacha.clear()
captacha_value = input(" **** Enter Captacha: ")
captacha.send_keys(captacha_value)

search_button = driver.find_element_by_id("searchEmployer")
search_button.click()

In [17]:
# extract captcha image
captcha_image = driver.find_element("xpath",'//*[@id="capImg"]')
cap = captcha_image.screenshot("captcha.png")

# store captcha image in a variable
captcha_image = driver.find_element("xpath",'//*[@id="capImg"]')
cap = captcha_image.screenshot_as_png

In [18]:
import io
from PIL import Image
cap = Image.open(io.BytesIO(cap))
# display image
cap.show()

In [28]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver, 10)
cap = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="capImg"]')))

In [29]:
cap = cap.screenshot("captcha.png")

In [26]:
workbook = Workbook()
worksheet = workbook.active
begin = 1
date1 = datetime.datetime(2022,6,1)

In [7]:
def create_tables(start_val):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    bold_font = Font(bold=True)

    table = soup.select_one('#tablecontainer3 table')
    rows = table.find_all('tr')
    worksheet.cell(row=start_val,column=1,value="Validity Status-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val+1):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)


    table = soup.select_one('#tablecontainer4 table')
    rows = table.find_all('tr')
    worksheet.cell(row=start_val + 7,column=1,value="Establishment Status-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val + 8):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)

    table = soup.select_one('#tablecontainer5 table')
    rows = table.find_all('tr')

    worksheet.cell(row=start_val + 14,column=1,value="Establishment Details-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val + 15):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            if cell_value == 'VERIFIED':
                fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
                worksheet.cell(row=row_index, column=col_index).fill = fill
            elif cell_value == 'REJECTED':
                fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
                worksheet.cell(row=row_index, column=col_index).fill = fill
            worksheet.cell(row=row_index, column=col_index, value=cell_value)
    try:
        table = soup.select_one('#tablecontainer12 table')
        rows = table.find_all('tr')

        worksheet.cell(row=start_val + 26,column=1,value="Additional Information-").font = bold_font
        for row_index, row in enumerate(rows, start=start_val + 27):
            columns = row.find_all('td')
            for col_index, column in enumerate(columns, start=1):
                cell_value = column.get_text()
                worksheet.cell(row=row_index, column=col_index, value=cell_value)

        row_color = 'ADD8E6'
        fill = PatternFill(start_color=row_color, end_color=row_color, fill_type='solid')

        row_number = start_val + 35
        row = worksheet[row_number]

        for cell in row:
            cell.fill = fill
    except (AttributeError, NoSuchElementException):
        worksheet.cell(row=start_val + 26,column=1,value="No Additional Information").font = bold_font

def payments_table(begin)->int:
    html_content = driver.page_source

    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.select_one('#table_pop_up')

    rows = table.find_all('tr')

    for row_index, row in enumerate(rows, start=begin):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            if col_index == 2:
                payment_date = datetime.datetime.strptime(cell_value,'%d-%b-%Y %H:%M:%S')
            if col_index == 4:
                wage_date = datetime.datetime.strptime(cell_value, '%b-%y')
                wage_month = wage_date.month
                wage_year = wage_date.year
                wage_day = wage_date.day
                if  wage_date >=  date1:
                    cell_value = wage_date.strftime("%b-%y")
                    worksheet.cell(row=row_index, column=col_index, value=cell_value)
                    next_month = (wage_month % 12) + 1
                    payment_due_date = datetime.datetime(wage_year, next_month, 16)
                    if payment_due_date < payment_date:
                        worksheet.cell(row=row_index, column=7,value="DELAYED")
                    else:
                        worksheet.cell(row=row_index, column=7,value="-")
                else:
                    begin -= 1
                    break
            worksheet.cell(row=row_index, column=col_index, value=cell_value)
        begin += 1
    return begin-1

def row_count():
    # count the number of rows in the table
    table = driver.find_element_by_id("example")
    rows = table.find_elements(By.TAG_NAME, "tr")
    print("Number of rows in the table: ", len(rows))
    return len(rows)

def table_click(row):
    links = driver.find_elements("xpath", '//*[@id="example"]/tbody/tr['+str(row)+']/td[5]/a')
    if links:
        links[0].click()

def heading(begin):
    bold_font = Font(bold=True)
    worksheet.cell(row=begin,column=1,value="TRRN").font = bold_font
    worksheet.cell(row=begin,column=2,value="Date of Credit").font = bold_font
    worksheet.cell(row=begin,column=3,value="Amount").font = bold_font
    worksheet.cell(row=begin,column=4,value="Wage Month").font = bold_font
    worksheet.cell(row=begin,column=5,value="No. of Employees").font = bold_font
    worksheet.cell(row=begin,column=6,value="ECR").font = bold_font
    worksheet.cell(row=begin,column=7,value="DELAY").font = bold_font

def header_2(begin):
    bold_font = Font(bold=True)
    worksheet.cell(row=begin,column=1,value="E_ID").font = bold_font
    worksheet.cell(row=begin,column=2,value="E_Name").font = bold_font
    worksheet.cell(row=begin,column=3,value="Address").font = bold_font
    worksheet.cell(row=begin,column=4,value="Office Name").font = bold_font


In [169]:
driver.switch_to.window(driver.window_handles[0])

In [20]:
table_click(1)

In [64]:
for i in range(1,2):
    row_len = 2
    for row in range(1, row_len):
        table_click(row)
        time.sleep(10)
        create_tables(begin)
        begin = begin + 37
        payments_link = driver.find_element("xpath",'//*[@id="tablecontainer3"]/div/a/u')
        payments_link.click()
        time.sleep(5)

        driver.switch_to.window(driver.window_handles[1])
        
        pages_in_payments_text = driver.find_element("xpath",'//*[@id="table_pop_up_info"]')
        pattern = r"(\d+)$"
        nos_pages_in_payments = re.search(pattern,pages_in_payments_text.text)

        if nos_pages_in_payments:
            number_of_pays = nos_pages_in_payments.group(1)
            print("Number of payment pages:",number_of_pays)
            heading(begin)
        for nos in range(1,int(number_of_pays)+1):
            begin = payments_table(begin)
            row_number = begin + 1
            next_button = driver.find_element("xpath",'//*[@id="table_pop_up_next"]')
            next_button.click()
        row_color = 'ADD8E6'
        fill = PatternFill(start_color=row_color, end_color=row_color, fill_type='solid')
        row = worksheet[row_number]

        for cell in row:
            cell.fill = fill
        time.sleep(10)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(3)
        begin = begin + 1

Number of payment pages: 13


In [21]:
same_pan = driver.find_element("xpath",'//*[@id="tablecontainer11"]/div/a')
same_pan.click()

In [22]:
driver.switch_to.window(driver.window_handles[1])

In [67]:
begin +=1
header_2(begin)

In [11]:
def same_pan_company(start_val)->int:
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.select_one('#table_pop_up_wrapper table')

    rows = table.find_all('tr')
    
    for row_index, row in enumerate(rows, start=start_val):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)
        start_val += 1

    return start_val

In [69]:
begin -=1

In [70]:
begin = same_pan_company(begin)

In [71]:
workbook.save(filename=COMPANY_NAME + '1.xlsx')

In [12]:
def create_tables_2(start_val):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    bold_font = Font(bold=True)

    table = soup.select_one('#tbPopUp_3 table')
    rows = table.find_all('tr')
    worksheet.cell(row=start_val,column=1,value="Validity Status-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val+1):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)


    table = soup.select_one('#tbPopUp_4 table')
    rows = table.find_all('tr')
    worksheet.cell(row=start_val + 7,column=1,value="Establishment Status-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val + 8):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)

    table = soup.select_one('#tbPopUp_5 table')
    rows = table.find_all('tr')

    worksheet.cell(row=start_val + 14,column=1,value="Establishment Details-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val + 15):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            if cell_value == 'VERIFIED':
                fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
                worksheet.cell(row=row_index, column=col_index).fill = fill
            elif cell_value == 'REJECTED':
                fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
                worksheet.cell(row=row_index, column=col_index).fill = fill
            worksheet.cell(row=row_index, column=col_index, value=cell_value)
    
    try:
        table = soup.select_one('#tbPopUp_12 table')
        rows = table.find_all('tr')
    except AttributeError:
        rows = []

    worksheet.cell(row=start_val + 26,column=1,value="Additional Information-").font = bold_font
    for row_index, row in enumerate(rows, start=start_val + 27):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=1):
            cell_value = column.get_text()
            worksheet.cell(row=row_index, column=col_index, value=cell_value)

    row_color = 'ADD8E6'
    fill = PatternFill(start_color=row_color, end_color=row_color, fill_type='solid')

    row_number = start_val + 35
    row = worksheet[row_number]

    for cell in row:
        cell.fill = fill

In [13]:
def count_pages()->int:
    total_pages = driver.find_element("xpath",'//*[@id="table_pop_up_info"]')
    page_count = int(total_pages.text.split()[-1])
    print("Number of pages:",page_count)
    return page_count

def table_click_2(row):
    E_ID = driver.find_elements("xpath", '//*[@id="table_pop_up"]/tbody/tr['+str(row)+']/td[1]/a')
    if E_ID:
        E_ID[0].click()
        return True
    
def row_count_2():
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.select_one('#table_pop_up_wrapper table')
    rows = table.find_all('tr')
    return len(rows)

In [23]:
pages = count_pages()
for i in range(1,pages):
    num_of_rows = row_count_2()
    for row in range(1,num_of_rows):
        table_click_2(row)
        time.sleep(5)
        create_tables_2(begin)
        begin += 37

        payments_link = driver.find_element("xpath",'//*[@id="tbPopUp_3"]/div/a/u')
        payments_link.click()
        time.sleep(3)

        driver.switch_to.window(driver.window_handles[2])
        try:
            pages_in_payments_text = driver.find_element("xpath",'//*[@id="table_pop_up_info"]')
            pattern = r"(\d+)$"
            nos_pages_in_payments = re.search(pattern,pages_in_payments_text.text)
            if nos_pages_in_payments:
                number_of_pays = nos_pages_in_payments.group(1)
                print("Number of payment pages:",number_of_pays)
                heading(begin)
            for nos in range(1,int(number_of_pays)+1):
                begin = payments_table(begin)
                row_number = begin + 1
                next_button = driver.find_element("xpath",'//*[@id="table_pop_up_next"]')
                next_button.click()
            row_color = 'ADD8E6'
            fill = PatternFill(start_color=row_color, end_color=row_color, fill_type='solid')
            row = worksheet[row_number]

            for cell in row:
                cell.fill = fill
            driver.close()
        except:
            driver.close()
        driver.switch_to.window(driver.window_handles[1])
        # time.sleep(3)
        begin += 2
        
    # next_button = driver.find_element("xpath",'//*[@id="table_pop_up_next"]')
    # next.click()
    # time.sleep(3)
        

Number of pages: 2
Number of payment pages: 13
Number of payment pages: 13
Number of payment pages: 7
Number of payment pages: 6
Number of payment pages: 10
Number of payment pages: 12
Number of payment pages: 25
Number of payment pages: 9
Number of payment pages: 2


In [ ]:
next_button = driver.find_element("xpath",'//*[@id="table_pop_up_next"]')
next.click()

In [67]:
driver.switch_to.window(driver.window_handles[0])

In [50]:
driver.switch_to.window(driver.window_handles[1])
payments_link = driver.find_element("xpath",'//*[@id="tbPopUp_3"]/div/a/u')
payments_link.click()
time.sleep(10)

driver.switch_to.window(driver.window_handles[2])

pages_in_payments_text = driver.find_element("xpath",'//*[@id="table_pop_up_info"]')
pattern = r"(\d+)$"
nos_pages_in_payments = re.search(pattern,pages_in_payments_text.text)
if nos_pages_in_payments:
    number_of_pays = nos_pages_in_payments.group(1)
    print("Number of payment pages:",number_of_pays)
    heading(begin)

for nos in range(1,int(number_of_pays)+1):
    begin = payments_table(begin)
    row_number = begin + 1
    next_button = driver.find_element("xpath",'//*[@id="table_pop_up_next"]')
    next_button.click()

Number of payment pages: 31


In [24]:
workbook.save(filename=str(COMPANY_NAME)+"2.xlsx")

In [25]:
from bs4 import BeautifulSoup
from openpyxl.styles import Font, PatternFill
import re
from openpyxl.styles import PatternFill

In [32]:
def get_table_rows(soup, table_id):
    try:
        table = soup.select_one(f'#{table_id} table')
        return table.find_all('tr')
    except AttributeError:
        return []

def write_to_worksheet(rows, start_row, start_col, worksheet, highlight_conditions=None):
    for row_index, row in enumerate(rows, start=start_row):
        columns = row.find_all('td')
        for col_index, column in enumerate(columns, start=start_col):
            cell_value = column.get_text()
            cell = worksheet.cell(row=row_index, column=col_index, value=cell_value)
            
            if highlight_conditions and cell_value in highlight_conditions:
                fill = PatternFill(start_color=highlight_conditions[cell_value], end_color=highlight_conditions[cell_value], fill_type='solid')
                cell.fill = fill

def create_tables_2(start_val, driver, worksheet):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    bold_font = Font(bold=True)

    table_data = [
        {'table_id': 'tbPopUp_3', 'title': 'Validity Status-', 'start_row': start_val},
        {'table_id': 'tbPopUp_4', 'title': 'Establishment Status-', 'start_row': start_val + 7},
        {'table_id': 'tbPopUp_5', 'title': 'Establishment Details-', 'start_row': start_val + 14, 'highlight_conditions': {'VERIFIED': '00FF00', 'REJECTED': 'FF0000'}},
        {'table_id': 'tbPopUp_12', 'title': 'Additional Information-', 'start_row': start_val + 26}
    ]

    # This will store the last row written to by the function
    last_row_written = 0

    for table_info in table_data:
        rows = get_table_rows(soup, table_info['table_id'])
        worksheet.cell(row=table_info['start_row'], column=1, value=table_info['title']).font = bold_font
        write_to_worksheet(rows, table_info['start_row'] + 1, 1, worksheet, table_info.get('highlight_conditions'))
        
        # Update the last row written to, if the current table went beyond the previous one
        last_row_written = max(last_row_written, table_info['start_row'] + len(rows))

    # Colorize the row after the last table
    colorize_row(worksheet[last_row_written + 1], 'ADD8E6')
    
    # Return the row number after the colorized row
    return last_row_written + 2

def colorize_row(row, color):
    fill = PatternFill(start_color=color, end_color=color, fill_type='solid')
    for cell in row:
        cell.fill = fill


In [33]:
def get_number_of_payment_pages(driver):
    pages_in_payments_text = driver.find_element("xpath",'//*[@id="table_pop_up_info"]')
    pattern = r"(\d+)$"
    nos_pages_in_payments = re.search(pattern, pages_in_payments_text.text)
    return int(nos_pages_in_payments.group(1)) if nos_pages_in_payments else 0

def handle_payment_pages(driver, start_row):
    number_of_pays = get_number_of_payment_pages(driver)
    print("Number of payment pages:", number_of_pays)
    heading(start_row)
    
    for _ in range(number_of_pays):
        start_row = payments_table(start_row)
        next_button = driver.find_element("xpath", '//*[@id="table_pop_up_next"]')
        next_button.click()

    colorize_row(worksheet[start_row + 1], 'ADD8E6')

def colorize_row(row, color):
    fill = PatternFill(start_color=color, end_color=color, fill_type='solid')
    for cell in row:
        cell.fill = fill

def main_process(driver, worksheet, begin, pages):
    for i in range(1, pages):
        num_of_rows = row_count_2()
        for row in range(1, num_of_rows):
            table_click_2(row)
            time.sleep(5)
            create_tables_2(begin,driver,worksheet)
            begin += 37

            payments_link = driver.find_element("xpath", '//*[@id="tbPopUp_3"]/div/a/u')
            payments_link.click()
            time.sleep(3)

            driver.switch_to.window(driver.window_handles[2])
            try:
                handle_payment_pages(driver, begin)
                driver.close()
            except Exception as e:
                print(f"Error encountered: {e}")
                driver.close()

            driver.switch_to.window(driver.window_handles[1])
            begin += 2

pages = count_pages()
main_process(driver, worksheet, begin, pages)

Number of pages: 2
Number of payment pages: 13
Number of payment pages: 13
Number of payment pages: 7
Number of payment pages: 6
Number of payment pages: 10
Number of payment pages: 12
Number of payment pages: 25
Number of payment pages: 9
Error encountered: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="table_pop_up_info"]"}
  (Session info: chrome=116.0.5845.111)

Number of payment pages: 2


In [34]:
workbook.save(filename=str(COMPANY_NAME)+"2.xlsx")